# 题目

> 给你一个字符串 `s` ，找到 `s` 中最长的回文子串。  
如果字符串的反序与原始字符串相同，则该字符串称为回文字符串。

# 方法一：动态规划

> 若一个字符串是回文串，则去掉其首位两个字符后仍为回文串。因此，设 $s[i,j]$ 为字符串 `s` 中第i位到第j位组成的子串。  
只有当 $s[i+1:j−1]$ 是回文串，并且s的第i和j个字母相同时， $s[i:j]$ 才会是回文串。  
基础情况为：长度为1的子串 $s[i,i]$ 必为回文串，长度为2的子串 $s[i,j]$ 在两个字母相同时为回文串。

## 复杂度

- 时间复杂度: $O(n^2)$，其中 $n$ 是字符串 `s` 的长度。

> 动态规划的状态总数为 $O(n^2)$ ，对于每个状态，我们需要转移的时间为 $O(1)$ 。

- 空间复杂度: $O(n^2)$，其中 $n$ 是字符串 `s` 的长度。

> 存储动态规划状态需要的空间。

## 代码

In [1]:
def longestPalindrome(s):
    n = len(s)
    if n < 2:
        return s
    
    max_len = 1
    begin = 0
    # dp[i][j] 表示 s[i..j] 是否是回文串
    dp = [[False] * n for _ in range(n)]  # 创建一个n×n的列表
    for i in range(n):
        dp[i][i] = True  # 所有长度为1的子串都为回文串，设为True
    
    # 递推开始
    # 先枚举子串长度
    for L in range(2, n + 1):  # L代表子串长度，至少为2，至多为n
        # 枚举左边界，左边界的上限设置可以宽松一些
        for i in range(n):
            # 由 L 和 i 可以确定右边界，即 j - i + 1 = L 得
            j = L + i - 1
            # 如果右边界越界，就可以退出当前循环
            if j >= n:
                break
                
            if s[i] != s[j]:
                dp[i][j] = False  # 若左右边界位置的字母不等，则s[i,j]不是回文串
            else:
                if j - i < 3:
                    dp[i][j] = True  # 若左右边界位置的字母相等，且s[i,j]长度小于4，则s[i,j],是回文串
                else:
                    dp[i][j] = dp[i + 1][j - 1]  # 若左右边界位置的字母相等，且s[i,j]长度大于等于4，则需要判断s[i + 1,j - 1]的情况
            
            # 只要 dp[i][j] == true 成立，就表示子串 s[i.j] 是回文，此时记录回文长度和起始位置
            if dp[i][j] and j - i + 1 > max_len:
                max_len = j - i + 1
                begin = i
    return s[begin:begin + max_len]

#### 测试一

In [2]:
s = "babad"
longestPalindrome(s)

#### 测试二

In [3]:
s = "cbbd"
longestPalindrome(s)

# 方法二：中心扩展算法

> 在动态规划中， $n^2$ 种状态都是通过边界情况转移得到的，边界情况即为子串长度为1或2的情况。  
因此可以枚举每种边界情况并不断向外扩展，直至两边字母不同，记录最大长度和回文中心的位置，选出其中长度最大的回文子串。

## 复杂度

- 时间复杂度: $O(n^2)$，其中 $n$ 是字符串 `s` 的长度。

> 动态规划的状态总数为 $O(n^2)$ ，对于每个状态，我们需要转移的时间为 $O(1)$ 。

- 空间复杂度: $O(1)$ 。

> 每次枚举时只需要记录最大长度和回文中心的位置。

# 方法三：Manacher 算法

> Manacher算法是中心扩展算法的进阶版，该算法引入新的概念：臂长length。假设一个回文中心的最大回文串长度为L，则L=2×length+1。在对新的位置i进行中心扩展时，可以利用之前以扩展的所有点的臂长信息。  
对于一个点j，假设其臂长为length，且有j+length>i，如下所示：  
e b **a** b **a** b **a** b e  
其中，回文中心j对应着第五个字母a，新的回文中心i对应第七个字母a，j的臂长为4。j+length>i表示i位于j的右臂长范围之内。  
此时，可以找到i关于j的对称点2×j-i，即第三个字母a，该回文中心的臂长为n=1。  
利用这些信息可以推断，回文中心i的臂长至少为min(j+length-i,n)，因此在扩展i时可以跳过这个最小臂长。  
其中，j+length-i表示i到以j为中心的最大回文串的最右端点。min(j+length-i,n)表示：i左右相邻的若干字母的对称情况应该与其对称点2×j-i相似，但相似的长度不能超过j为中心的最大回文串覆盖的范围，因为若超过了，则不能保证i与其对称点的相邻点是一一对应的。例如，若j+length-i=1<n=2，则只能保证点i的左右各一个点是对称的，再向外扩展时超出了j的臂长范围，则不一定是回文串（因为点i向右两个点i+2和对称点2×j-i向左两个点2×j-i-2不相等）。  
**只需要在中心扩展法的过程中记录右臂在最右边的回文字符串，将其中心作为j，在计算过程中就能最大限度地避免重复计算。**  
可以通过一个特别的操作将奇偶数的情况统一起来：我们向字符串的头尾以及每两个字符中间添加一个特殊字符 `#` ，比如字符串 `aaba` 处理后会变成 `#a#a#b#a#` 。那么原先长度为偶数的回文字符串 `aa` 会变成长度为奇数的回文字符串 `#a#a#` ，而长度为奇数的回文字符串 `aba` 会变成长度仍然为奇数的回文字符串 `#a#b#a#` ，我们就不需要再考虑长度为偶数的回文字符串了。  
注意这里的特殊字符不需要是没有出现过的字母，我们可以使用任何一个字符来作为这个特殊字符。这是因为，当我们只考虑长度为奇数的回文字符串时，每次我们比较的两个字符奇偶性一定是相同的，所以原来字符串中的字符不会与插入的特殊字符互相比较，不会因此产生问题。

## 复杂度

- 时间复杂度: $O(n)$，其中 $n$ 是字符串 `s` 的长度。

> 对于每个位置，扩展要么从当前的最右侧臂长 `right` 开始，要么只会进行一步（因为已有的信息已经帮助其略过重复的步骤），而 `right` 最多向右走 $O(n)$ 步，因此算法的复杂度为 $O(n)$ 。

- 空间复杂度: $O(n)$，其中 $n$ 是字符串 `s` 的长度。

> 需要 $O(n)$ 的空间记录每个位置的臂长。

## 代码

In [4]:
# 用于扩展回文串
def expand(s, left, right):
    while left >= 0 and right < len(s) and s[left] == s[right]:  # 若回文串左右端点都没有超过字符串的边界且彼此相等
        # 将左右端点向外扩展，直至左右端点不等
        left -= 1
        right += 1
    return (right - left - 2) // 2  # 返回当前回文中心的臂长

def longestPalindrome(s):
    end, start = -1, 0  # 分别记录最长回文串的右端点和左端点
    s = '#' + '#'.join(list(s)) + '#'  # 加入特殊字符处理偶数长度的回文串
    arm_len = []  # 记录每个回文中心的臂长
    right = -1  # 前一个回文中心的右端点
    j = -1  # 前一个回文中心
    for i in range(len(s)):
        
        if right >= i:  # 若i的位置仍在上一个回文中心j的臂长范围之内，则可以快速计算i的臂长
            i_sym = 2 * j - i
            min_arm_len = min(arm_len[i_sym], right - i)
            cur_arm_len = expand(s, i - min_arm_len, i + min_arm_len)
        else:  # 若i的位置在上一个回文中心j的臂长范围之外，则只能从回文中心开始计算臂长
            cur_arm_len = expand(s, i, i)
        
        arm_len.append(cur_arm_len)  #记录当前回文中心i的臂长
        
        if i + cur_arm_len > right:  # 若当前回文中心i的回文串右端点已超过上一个右端点，则需要更新right值
            j = i  # j表示前一个回文中心
            right = i + cur_arm_len  # 将right值更新为当前点的回文串右端点
        if 2 * cur_arm_len + 1 > end - start:  # 若当前回文串的长度大于最大回文串的长度，则更新end和start
            start = i - cur_arm_len
            end = i + cur_arm_len
    return s[start+1:end+1:2]  # 返回最长回文串，注意要跳过“#”

#### 测试一

In [5]:
s = "babad"
longestPalindrome(s)

#### 测试二

In [6]:
s = "cbbd"
longestPalindrome(s)